In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# save filepath 
home_data_file_path = '/Users/balqeesjabri/Downloads/train.csv'
# read the data and store data in DataFrame home_data
home_data = pd.read_csv(home_data_file_path)

In [ ]:
home_data.head()

Based on the charatistics of the dataset I have concluded that:
1. It is a supervised problem since the dataset includes labeled examples where each data point has a corresponding target variable( house price in this case)
2. The target variable is continuous (i.e numerical value house price), therefore, it is a regression problem

In [ ]:
# print a summary of the data in Melbourne data
summary_stats = home_data.describe()
print(summary_stats)

based on the above output:
I will utilize it to try and identify the outliers
by looking into features that have: (min,max,mean, 25th and 75th percentile) == 0

In [ ]:
# Create lists to store the features with specific conditions
features_zero_min = []
features_zero_max = []
features_zero_25th_percentile = []

# Filter only numeric columns
numeric_data = home_data.select_dtypes(include=[float, int])

# Check for features with zero minimum
features_zero_min = numeric_data.columns[numeric_data.min() == 0].tolist()

# Check for features with zero maximum
features_zero_max = numeric_data.columns[numeric_data.max() == 0].tolist()

# Check for features with zero at the 25th percentile
percentile_25_zeros = numeric_data.columns[numeric_data.quantile(0.25) == 0].tolist()

# Check for features with zero at the 75th percentile
percentile_75_zeros = numeric_data.columns[numeric_data.quantile(0.75) == 0].tolist()

# Find features that satisfy both conditions
selected_features = list(set(features_zero_min) & set(percentile_25_zeros) & set(percentile_75_zeros))
print("Features with zero minimum:", features_zero_min)
print("Features with zero maximum:", features_zero_max)
print("Features with zero at the 25th percentile:", percentile_25_zeros)
print('Features',selected_features)

In [ ]:
# CORRELATION MATRIX

# Filter only numerical columns for correlation calculation
numerical_data = home_data.select_dtypes(include=[float, int])

# Compute the correlation matrix
correlation_matrix = numerical_data.corr()

# Create the heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(data=correlation_matrix, annot=False, cmap='coolwarm')
plt.title("Correlation Matrix Heatmap")
plt.show()

FROM THIS HEATMAP:
Overallqualities feature has the highest correlation, followed by,
1. GrLivArea
2. GarageCars
3. GarageArea

In [ ]:
# target
y = home_data.SalePrice
X = home_data.drop(['SalePrice'], axis=1)
# Divide data into training and validation subsets
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X, y,
                                                                random_state=0)

In [ ]:
# is used to see the unique values of the target variable y (in this case, SalePrice)
print("Classes in the target variable:", np.unique(y))
avg_price = np.mean(y)
# will come in handy to see how far off is metric score
print(avg_price)

In [ ]:
# Select categorical columns
categorical_cols = [colname for colname in X_train_full.columns if X_train_full[colname].dtype == "object"]

# Select numerical columns
numerical_cols = [colname for colname in X_train_full.columns if X_train_full[colname].dtype in ['int64', 'float64']]

# Keep selected columns only
my_cols = categorical_cols + numerical_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()

In [ ]:
# THIS STEP IS CALLED PREPROCESSING DATA
# look into missing columns and see if its missingness holds value
cols_with_missing = [col for col in X_train.columns if X_train[col].isnull().any()]
# selected_cols_with_missing = cols_with_missing.drop(['BsmtQual'], ['BsmtCond'], ['BsmtExposure'], ['BsmtFinType1'], ['BsmtFinType2'], ['GarageType'])
print(cols_with_missing)

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder

# Create transformers for numerical and categorical features
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1))
])

# Create a ColumnTransformer to apply different transformers to feature types
preprocessor = ColumnTransformer(transformers=[
    ('num', numerical_transformer, numerical_cols),
    ('cat', categorical_transformer, categorical_cols)
])


In [ ]:
# Step 2: Define the Model
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(n_estimators=250, random_state=55)

1. n_estimators=100, random_state=0:  rmse: 31320.64408937722
2. n_estimators=300, random_state=0:  rmse: 31240.181525443895
3. n_estimators=300, random_state=42: rmse: 30848.3330883714
4. n_estimators=400, random_state=42  rmse: 30675.564025063748
5. n_estimators=400, random_state=55  rmse: 30068.437446816566
    - BEST SCORE:
    - n_estimators=250, random_state=55  rmse: 29866.344956563484

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Create a pipline 
my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', model)
                             ])
# This line fits the pipeline to the training data,
my_pipeline.fit(X_train, y_train)

# Preprocessing of validation data, get predictions
preds = my_pipeline.predict(X_valid)

# Evaluate the model
score = mean_absolute_error(y_valid, preds)
rmse = np.sqrt(mean_squared_error(y_valid, preds))
print('score', score)
print('rmse:', rmse)